In [61]:
with open(r'C:\Users\IT\Documents\UCL\pyssem\scenario-properties-elliptical.pkl', 'rb') as f:
    scen_properties = pickle.load(f)

all_elliptical_collision_species = scen_properties.collision_pairs

debris_species = [species for species in scen_properties.species['debris']]

# Mass Binning
binC_mass = np.zeros(len(debris_species))
binE_mass = np.zeros(2 * len(debris_species))
binW_mass = np.zeros(len(debris_species))
LBgiven = scen_properties.LC

for index, debris in enumerate(debris_species):
    binC_mass[index] = debris.mass
    binE_mass[2 * index: 2 * index + 2] = [debris.mass_lb, debris.mass_ub]
    binW_mass[index] = debris.mass_ub - debris.mass_lb

binE_mass = np.unique(binE_mass)

# Eccentricity Binning, multiple debris species will have the same eccentricity bins
binE_ecc = debris_species[0].eccentricity_bins
binE_ecc = np.sort(binE_ecc)
# Calculate the midpoints
binE_ecc = (binE_ecc[:-1] + binE_ecc[1:]) / 2
# Create bin edges starting at 0 and finishing at 1
binE_ecc = np.concatenate(([0], binE_ecc, [1]))

    
def evolve_bins(scen_properties, m1, m2, r1, r2, v1, v2, binE_mass, binE_ecc, collision_index, n_shells=0):
    
    # Need to now follow the NASA SBM route, first we need to create p1_in and p2_in
    #  Parameters:
    # - ep: Epoch
    # - p1_in: Array containing [mass, radius, r_x, r_y, r_z, v_x, v_y, v_z, object_class]
    # - p2_in: Array containing [mass, radius, r_x, r_y, r_z, v_x, v_y, v_z, object_class]
    # - param: Dictionary containing parameters like 'max_frag', 'mu', 'req', 'maxID', etc.
    # - LB: Lower bound for fragment sizes (meters)

    p1_in = np.array([
        1250.0,  # mass in kg
        4.0,     # radius in meters
        2372.4,  # r_x in km, 1000 km
        2743.1,  # r_y in km
        6224.8,  # r_z in km
        -5.5,    # v_x in km/s
        -3.0,    # v_y in km/s
        3.8,     # v_z in km/s
        1      # object_class (dimensionless)
    ])

    p2_in = np.array([
        6.0,     # mass in kg
        0.1,     # radius in meters
        2372.4,  # r_x in km
        2743.1,  # r_y in km
        6224.8,  # r_z in km
        3.2,     # v_x in km/s
        5.4,     # v_y in km/s
        -3.9,    # v_z in km/s
        1      # object_class (dimensionless)
    ])

    param = {
        'req': 6.3781e+03,
        'mu': 3.9860e+05,
        'j2': 0.0011,
        'max_frag': float('inf'),  # Inf in MATLAB translates to float('inf') in Python
        'maxID': 0,
        'density_profile': 'static'
    }
    
    a = scen_properties.HMid[collision_index] 

    # up to correct mass too
    if m1 < m2:
        m1, m2 = m2, m1
        r1, r2 = r2, r1

    p1_in[0], p2_in[0] = m1, m2 
    p1_in[1], p2_in[1] = r1, r2

    # remove a from r_x from both p1_in and p2_in
    p1_in[2] = p1_in[2] - a
    p2_in[2] = p2_in[2] - a
        
    LB = 0.1

    debris1, debris2, isCatastrophic = frag_col_SBM_vec_lc2(0, p1_in, p2_in, param, LB)

    print(len(debris1), len(debris2))


def process_elliptical_collision_pair(args):
    """
    A similar function to the process_species_pair, apart from it as the shells are already defined and the velocity, 
    you are able to calculate evolve bins just once. 

    """
    i, collision_pair, scen_properties, debris_species, binE_mass, binE_ecc, LBgiven = args
    m1, m2 = collision_pair.species1.mass, collision_pair.species2.mass
    r1, r2 = collision_pair.species1.radius, collision_pair.species2.radius


    # there needs to be some changes here to account for the fact that the shells are already defined
    # set fragment_spreading to True
    v1 = collision_pair.species1.velocity_per_shells[collision_pair.shell_index][collision_pair.shell_index]
    v2 = collision_pair.species2.velocity_per_shells[collision_pair.shell_index][collision_pair.shell_index]

    # This the time per shell for each species - the output still assumes that it is always there, so you need to divide
    t1 = collision_pair.species1.time_per_shells[collision_pair.shell_index][collision_pair.shell_index]
    t2 = collision_pair.species2.time_per_shells[collision_pair.shell_index][collision_pair.shell_index]
    min_TIS = min(t1, t2)
    print(m1, m2, min_TIS)

    if m1 < 1 or m2 < 1:
        fragments = None
    else:
        fragments = evolve_bins(scen_properties, m1, m2, r1, r2, v1, v2, binE_mass, binE_ecc, collision_pair.shell_index, n_shells=scen_properties.n_shells)    
    
    if fragments is not None:
        collision_pair.fragments = fragments # * min_TIS
    else: 
        collision_pair.fragments = fragments

    return collision_pair



for i, species_pair in enumerate(tqdm(all_elliptical_collision_species, desc="Processing species pairs")):
    for shell_collision in tqdm(species_pair.collision_pair_by_shell, desc="Processing shell collisions", leave=False):
        
        # Process each shell-specific collision and append result to collision_processed list
        gamma = process_elliptical_collision_pair((i, shell_collision, scen_properties, debris_species, binE_mass, binE_ecc, LBgiven))
        species_pair.collision_processed.append(gamma)

Processing species pairs:   0%|          | 0/10 [00:00<?, ?it/s]

0.00141372 2 1.0
0.00141372 2 1.0
0.00141372 2 0.964
0.00141372 2 0.964
0.00141372 2 0.936
0.00141372 2 0.936
0.00141372 2 0.916
0.00141372 2 0.916
0.00141372 2 0.902
0.00141372 2 0.902
0.00141372 2 0.888
0.00141372 2 0.888
0.00141372 2 0.878
0.00141372 2 0.878
0.00141372 2 0.868
0.00141372 2 0.868
0.00141372 2 0.858
0.00141372 2 0.858
0.00141372 2 0.85


0.00141372 20 1.0
0.00141372 20 1.0
0.00141372 20 0.964
0.00141372 20 0.964
0.00141372 20 0.936
0.00141372 20 0.936
0.00141372 20 0.916
0.00141372 20 0.916
0.00141372 20 0.902
0.00141372 20 0.902
0.00141372 20 0.888
0.00141372 20 0.888
0.00141372 20 0.878
0.00141372 20 0.878
0.00141372 20 0.868
0.00141372 20 0.868
0.00141372 20 0.858
0.00141372 20 0.858
0.00141372 20 0.85


2 0.00141372 1.0
2 0.00141372 1.0
2 0.00141372 0.964
2 0.00141372 0.964
2 0.00141372 0.936
2 0.00141372 0.936
2 0.00141372 0.916
2 0.00141372 0.916
2 0.00141372 0.902
2 0.00141372 0.902
2 0.00141372 0.888
2 0.00141372 0.888
2 0.00141372 0.878
2 0.00141372 0.878
2 0.00141372 0.868
2 0.00141372 0.868
2 0.00141372 0.858
2 0.00141372 0.858
2 0.00141372 0.85


2 2 1.0
Number of fragments: 14



100%|██████████| 10/10 [00:00<?, ?it/s]

Processing species pairs:  40%|████      | 4/10 [00:00<00:00, 39.45it/s]


10 4
2 2 1.0
Number of fragments: 17


TypeError: 'numpy.float64' object is not iterable